# Lesson A7: Unsafe Rust & FFI

**Duration**: 135-150 minutes  
**Stage**: Advanced (Mastery)

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Understand when and why to use unsafe Rust
2. Work with raw pointers and unsafe operations
3. Implement Foreign Function Interface (FFI)
4. Maintain safety invariants in unsafe code
5. Debug and test unsafe code effectively

---

## 📋 Prerequisite Review

**Quick Check**: From our previous lessons:

1. How do declarative macros work with macro_rules!?
2. What is macro hygiene?
3. How do you handle repetition in macros?
4. What are the benefits of metaprogramming?

**Answers**: 1) Pattern-based code generation, 2) Automatic variable scoping, 3) $()* syntax for repetition, 4) Code generation, DSLs, compile-time computation

---

## 🧠 Key Concepts

### Unsafe Rust Capabilities

**Raw Pointers**: *const T and *mut T for direct memory access
**Unsafe Functions**: Functions that require unsafe blocks to call
**Unsafe Traits**: Traits that have safety requirements
**Static Mutable Variables**: Global mutable state
**Union Types**: C-style unions for memory layout control

### Safety Invariants

- **Memory Safety**: Prevent use-after-free, double-free, buffer overflows
- **Thread Safety**: Ensure data races don't occur
- **Type Safety**: Maintain type system guarantees
- **Lifetime Safety**: Respect borrowing rules manually

---

## 🔬 Live Code Exploration

### Raw Pointers and Basic Unsafe Operations

In [ ]:
// Raw pointers and basic unsafe operations

use std::ptr;

fn raw_pointers_demo() {
    println!("=== Raw Pointers and Unsafe Operations ===");
    
    // Creating raw pointers from references
    let mut x = 42;
    let r1 = &x as *const i32;  // Immutable raw pointer
    let r2 = &mut x as *mut i32; // Mutable raw pointer
    
    println!("Raw pointers created:");
    println!("  r1 (const): {:p}", r1);
    println!("  r2 (mut):   {:p}", r2);
    
    // Dereferencing raw pointers (requires unsafe)
    unsafe {
        println!("\nDereferencing raw pointers:");
        println!("  *r1 = {}", *r1);
        println!("  *r2 = {}", *r2);
        
        // Modifying through mutable raw pointer
        *r2 = 100;
        println!("  After *r2 = 100: x = {}", x);
    }
    
    // Creating raw pointers from arbitrary addresses (dangerous!)
    let address = 0x12345usize;
    let r3 = address as *const i32;
    println!("\nRaw pointer from address: {:p}", r3);
    // Note: Dereferencing this would likely crash!
    
    // Null pointers
    let null_ptr: *const i32 = ptr::null();
    let null_mut_ptr: *mut i32 = ptr::null_mut();
    
    println!("\nNull pointers:");
    println!("  null_ptr.is_null(): {}", null_ptr.is_null());
    println!("  null_mut_ptr.is_null(): {}", null_mut_ptr.is_null());
}

// Unsafe functions
unsafe fn dangerous_function() {
    println!("This is an unsafe function!");
}

// Safe wrapper around unsafe code
fn safe_wrapper(slice: &[i32], index: usize) -> Option<i32> {
    if index < slice.len() {
        unsafe {
            // We've checked bounds, so this is safe
            Some(*slice.get_unchecked(index))
        }
    } else {
        None
    }
}

// Custom unsafe trait
unsafe trait UnsafeTrait {
    fn unsafe_method(&self);
}

// Implementing unsafe trait
struct SafeStruct {
    value: i32,
}

unsafe impl UnsafeTrait for SafeStruct {
    fn unsafe_method(&self) {
        println!("Unsafe method called with value: {}", self.value);
    }
}

fn unsafe_operations_demo() {
    println!("\n=== Unsafe Functions and Traits ===");
    
    // Calling unsafe function
    unsafe {
        dangerous_function();
    }
    
    // Using safe wrapper
    let numbers = vec![1, 2, 3, 4, 5];
    println!("\nSafe wrapper results:");
    println!("  Index 2: {:?}", safe_wrapper(&numbers, 2));
    println!("  Index 10: {:?}", safe_wrapper(&numbers, 10));
    
    // Using unsafe trait
    let safe_struct = SafeStruct { value: 42 };
    safe_struct.unsafe_method(); // This is safe because we implemented it correctly
}

raw_pointers_demo();
unsafe_operations_demo();

### Memory Management and Custom Allocators

In [ ]:
// Memory management and custom data structures

use std::alloc::{alloc, dealloc, Layout};
use std::ptr;
use std::mem;

// Custom vector implementation using unsafe code
struct UnsafeVec<T> {
    ptr: *mut T,
    len: usize,
    capacity: usize,
}

impl<T> UnsafeVec<T> {
    fn new() -> Self {
        UnsafeVec {
            ptr: ptr::null_mut(),
            len: 0,
            capacity: 0,
        }
    }
    
    fn with_capacity(capacity: usize) -> Self {
        let mut vec = UnsafeVec::new();
        if capacity > 0 {
            vec.grow(capacity);
        }
        vec
    }
    
    fn grow(&mut self, new_capacity: usize) {
        if new_capacity <= self.capacity {
            return;
        }
        
        let layout = Layout::array::<T>(new_capacity).unwrap();
        
        unsafe {
            let new_ptr = if self.capacity == 0 {
                alloc(layout) as *mut T
            } else {
                // In a real implementation, we'd use realloc
                let new_ptr = alloc(layout) as *mut T;
                
                // Copy existing elements
                ptr::copy_nonoverlapping(self.ptr, new_ptr, self.len);
                
                // Deallocate old memory
                let old_layout = Layout::array::<T>(self.capacity).unwrap();
                dealloc(self.ptr as *mut u8, old_layout);
                
                new_ptr
            };
            
            self.ptr = new_ptr;
            self.capacity = new_capacity;
        }
    }
    
    fn push(&mut self, value: T) {
        if self.len == self.capacity {
            let new_capacity = if self.capacity == 0 { 1 } else { self.capacity * 2 };
            self.grow(new_capacity);
        }
        
        unsafe {
            ptr::write(self.ptr.add(self.len), value);
        }
        
        self.len += 1;
    }
    
    fn pop(&mut self) -> Option<T> {
        if self.len == 0 {
            None
        } else {
            self.len -= 1;
            unsafe {
                Some(ptr::read(self.ptr.add(self.len)))
            }
        }
    }
    
    fn get(&self, index: usize) -> Option<&T> {
        if index < self.len {
            unsafe {
                Some(&*self.ptr.add(index))
            }
        } else {
            None
        }
    }
    
    fn get_mut(&mut self, index: usize) -> Option<&mut T> {
        if index < self.len {
            unsafe {
                Some(&mut *self.ptr.add(index))
            }
        } else {
            None
        }
    }
    
    fn len(&self) -> usize {
        self.len
    }
    
    fn capacity(&self) -> usize {
        self.capacity
    }
    
    fn is_empty(&self) -> bool {
        self.len == 0
    }
}

impl<T> Drop for UnsafeVec<T> {
    fn drop(&mut self) {
        // Drop all elements
        while let Some(_) = self.pop() {}
        
        // Deallocate memory
        if self.capacity > 0 {
            unsafe {
                let layout = Layout::array::<T>(self.capacity).unwrap();
                dealloc(self.ptr as *mut u8, layout);
            }
        }
    }
}

// Union example (C-style union)
#[repr(C)]
union IntOrFloat {
    i: i32,
    f: f32,
}

impl IntOrFloat {
    fn new_int(value: i32) -> Self {
        IntOrFloat { i: value }
    }
    
    fn new_float(value: f32) -> Self {
        IntOrFloat { f: value }
    }
    
    unsafe fn as_int(&self) -> i32 {
        self.i
    }
    
    unsafe fn as_float(&self) -> f32 {
        self.f
    }
}

// Static mutable variable (requires unsafe to access)
static mut GLOBAL_COUNTER: i32 = 0;

fn increment_global_counter() {
    unsafe {
        GLOBAL_COUNTER += 1;
    }
}

fn get_global_counter() -> i32 {
    unsafe {
        GLOBAL_COUNTER
    }
}

fn memory_management_demo() {
    println!("\n=== Memory Management and Custom Data Structures ===");
    
    // Custom vector demonstration
    let mut vec = UnsafeVec::with_capacity(2);
    println!("Created UnsafeVec with capacity: {}", vec.capacity());
    
    // Push elements
    vec.push("Hello");
    vec.push("World");
    vec.push("!"); // This will trigger growth
    
    println!("\nAfter pushing 3 elements:");
    println!("  Length: {}", vec.len());
    println!("  Capacity: {}", vec.capacity());
    
    // Access elements
    for i in 0..vec.len() {
        if let Some(value) = vec.get(i) {
            println!("  vec[{}] = {}", i, value);
        }
    }
    
    // Pop elements
    while let Some(value) = vec.pop() {
        println!("Popped: {}", value);
    }
    
    println!("Vector is now empty: {}", vec.is_empty());
    
    // Union demonstration
    let int_val = IntOrFloat::new_int(42);
    let float_val = IntOrFloat::new_float(3.14);
    
    unsafe {
        println!("\nUnion demonstration:");
        println!("  int_val as int: {}", int_val.as_int());
        println!("  int_val as float: {}", int_val.as_float()); // Undefined behavior!
        println!("  float_val as float: {}", float_val.as_float());
        println!("  float_val as int: {}", float_val.as_int()); // Undefined behavior!
    }
    
    // Static mutable variable
    println!("\nGlobal counter demonstration:");
    println!("  Initial value: {}", get_global_counter());
    
    increment_global_counter();
    increment_global_counter();
    increment_global_counter();
    
    println!("  After 3 increments: {}", get_global_counter());
}

memory_management_demo();

### Foreign Function Interface (FFI)

In [ ]:
// Foreign Function Interface (FFI) examples

use std::ffi::{CStr, CString};
use std::os::raw::{c_char, c_int, c_void};

// External C functions (these would normally be in a C library)
extern "C" {
    // Standard C library functions
    fn strlen(s: *const c_char) -> usize;
    fn strcmp(s1: *const c_char, s2: *const c_char) -> c_int;
    fn malloc(size: usize) -> *mut c_void;
    fn free(ptr: *mut c_void);
}

// Rust functions that can be called from C
#[no_mangle]
pub extern "C" fn rust_add(a: c_int, b: c_int) -> c_int {
    a + b
}

#[no_mangle]
pub extern "C" fn rust_string_length(s: *const c_char) -> usize {
    if s.is_null() {
        return 0;
    }
    
    unsafe {
        CStr::from_ptr(s).to_bytes().len()
    }
}

#[no_mangle]
pub extern "C" fn rust_process_array(arr: *mut c_int, len: usize) {
    if arr.is_null() {
        return;
    }
    
    unsafe {
        let slice = std::slice::from_raw_parts_mut(arr, len);
        for item in slice {
            *item *= 2; // Double each element
        }
    }
}

// Safe wrapper for C string operations
struct CStringWrapper {
    ptr: *mut c_char,
}

impl CStringWrapper {
    fn new(s: &str) -> Result<Self, std::ffi::NulError> {
        let c_string = CString::new(s)?;
        let len = c_string.as_bytes_with_nul().len();
        
        unsafe {
            let ptr = malloc(len) as *mut c_char;
            if ptr.is_null() {
                panic!("Failed to allocate memory");
            }
            
            std::ptr::copy_nonoverlapping(
                c_string.as_ptr(),
                ptr,
                len,
            );
            
            Ok(CStringWrapper { ptr })
        }
    }
    
    fn as_ptr(&self) -> *const c_char {
        self.ptr
    }
    
    fn to_string(&self) -> Result<String, std::str::Utf8Error> {
        unsafe {
            CStr::from_ptr(self.ptr).to_str().map(|s| s.to_owned())
        }
    }
    
    fn len(&self) -> usize {
        unsafe {
            strlen(self.ptr)
        }
    }
}

impl Drop for CStringWrapper {
    fn drop(&mut self) {
        unsafe {
            free(self.ptr as *mut c_void);
        }
    }
}

// Callback function type for C
type Callback = extern "C" fn(c_int) -> c_int;

// Function that takes a callback
#[no_mangle]
pub extern "C" fn rust_apply_callback(arr: *mut c_int, len: usize, callback: Callback) {
    if arr.is_null() {
        return;
    }
    
    unsafe {
        let slice = std::slice::from_raw_parts_mut(arr, len);
        for item in slice {
            *item = callback(*item);
        }
    }
}

// Example callback functions
extern "C" fn square(x: c_int) -> c_int {
    x * x
}

extern "C" fn increment(x: c_int) -> c_int {
    x + 1
}

// Opaque type for C API
#[repr(C)]
pub struct OpaqueHandle {
    _private: [u8; 0],
}

// Internal implementation
struct InternalData {
    value: i32,
    name: String,
}

#[no_mangle]
pub extern "C" fn create_handle(value: c_int, name: *const c_char) -> *mut OpaqueHandle {
    if name.is_null() {
        return std::ptr::null_mut();
    }
    
    unsafe {
        let name_str = match CStr::from_ptr(name).to_str() {
            Ok(s) => s.to_owned(),
            Err(_) => return std::ptr::null_mut(),
        };
        
        let data = Box::new(InternalData {
            value,
            name: name_str,
        });
        
        Box::into_raw(data) as *mut OpaqueHandle
    }
}

#[no_mangle]
pub extern "C" fn get_handle_value(handle: *const OpaqueHandle) -> c_int {
    if handle.is_null() {
        return -1;
    }
    
    unsafe {
        let data = &*(handle as *const InternalData);
        data.value
    }
}

#[no_mangle]
pub extern "C" fn destroy_handle(handle: *mut OpaqueHandle) {
    if !handle.is_null() {
        unsafe {
            let _ = Box::from_raw(handle as *mut InternalData);
        }
    }
}

fn ffi_demo() {
    println!("\n=== Foreign Function Interface (FFI) ===");
    
    // Using C string functions
    let rust_string = "Hello, FFI!";
    let c_string = CString::new(rust_string).unwrap();
    
    unsafe {
        let len = strlen(c_string.as_ptr());
        println!("C strlen result: {}", len);
        println!("Rust len: {}", rust_string.len());
    }
    
    // String comparison
    let str1 = CString::new("hello").unwrap();
    let str2 = CString::new("hello").unwrap();
    let str3 = CString::new("world").unwrap();
    
    unsafe {
        let cmp1 = strcmp(str1.as_ptr(), str2.as_ptr());
        let cmp2 = strcmp(str1.as_ptr(), str3.as_ptr());
        
        println!("\nString comparison:");
        println!("  'hello' vs 'hello': {}", cmp1);
        println!("  'hello' vs 'world': {}", cmp2);
    }
    
    // C string wrapper
    let wrapper = CStringWrapper::new("Wrapped string").unwrap();
    println!("\nC string wrapper:");
    println!("  Length: {}", wrapper.len());
    println!("  Content: {}", wrapper.to_string().unwrap());
    
    // Array processing
    let mut numbers = [1, 2, 3, 4, 5];
    println!("\nArray before processing: {:?}", numbers);
    
    rust_process_array(numbers.as_mut_ptr(), numbers.len());
    println!("Array after doubling: {:?}", numbers);
    
    // Callback functions
    let mut values = [1, 2, 3, 4, 5];
    println!("\nCallback demonstration:");
    println!("  Original: {:?}", values);
    
    rust_apply_callback(values.as_mut_ptr(), values.len(), square);
    println!("  After square: {:?}", values);
    
    rust_apply_callback(values.as_mut_ptr(), values.len(), increment);
    println!("  After increment: {:?}", values);
    
    // Opaque handle
    let name = CString::new("Test Handle").unwrap();
    let handle = create_handle(42, name.as_ptr());
    
    if !handle.is_null() {
        let value = get_handle_value(handle);
        println!("\nOpaque handle value: {}", value);
        destroy_handle(handle);
        println!("Handle destroyed");
    }
    
    println!("\n💡 FFI Safety Tips:");
    println!("  - Always check for null pointers");
    println!("  - Validate string encoding (UTF-8 vs C strings)");
    println!("  - Match memory allocation/deallocation");
    println!("  - Use repr(C) for struct layout compatibility");
    println!("  - Handle errors gracefully across language boundaries");
}

ffi_demo();

---

## 🎯 Guided Practice

### Exercise 1: Safe Unsafe Code Patterns

Create safe abstractions around unsafe code.

In [ ]:
// TODO: Complete the safe unsafe code patterns

use std::ptr;
use std::mem;
use std::alloc::{alloc, dealloc, Layout};

// Safe ring buffer implementation using unsafe code
struct RingBuffer<T> {
    data: *mut T,
    capacity: usize,
    head: usize,
    tail: usize,
    len: usize,
}

impl<T> RingBuffer<T> {
    fn new(capacity: usize) -> Self {
        assert!(capacity > 0, "Capacity must be greater than 0");
        
        let layout = Layout::array::<T>(capacity).unwrap();
        let data = unsafe { alloc(layout) as *mut T };
        
        if data.is_null() {
            panic!("Failed to allocate memory for ring buffer");
        }
        
        RingBuffer {
            data,
            capacity,
            head: 0,
            tail: 0,
            len: 0,
        }
    }
    
    fn push(&mut self, value: T) -> Result<(), T> {
        if self.len == self.capacity {
            return Err(value); // Buffer is full
        }
        
        unsafe {
            ptr::write(self.data.add(self.tail), value);
        }
        
        self.tail = (self.tail + 1) % self.capacity;
        self.len += 1;
        
        Ok(())
    }
    
    fn pop(&mut self) -> Option<T> {
        if self.len == 0 {
            return None;
        }
        
        let value = unsafe {
            ptr::read(self.data.add(self.head))
        };
        
        self.head = (self.head + 1) % self.capacity;
        self.len -= 1;
        
        Some(value)
    }
    
    fn peek(&self) -> Option<&T> {
        if self.len == 0 {
            None
        } else {
            unsafe {
                Some(&*self.data.add(self.head))
            }
        }
    }
    
    fn len(&self) -> usize {
        self.len
    }
    
    fn capacity(&self) -> usize {
        self.capacity
    }
    
    fn is_empty(&self) -> bool {
        self.len == 0
    }
    
    fn is_full(&self) -> bool {
        self.len == self.capacity
    }
    
    // Safe iterator implementation
    fn iter(&self) -> RingBufferIter<T> {
        RingBufferIter {
            buffer: self,
            index: 0,
        }
    }
}

impl<T> Drop for RingBuffer<T> {
    fn drop(&mut self) {
        // Drop all remaining elements
        while let Some(_) = self.pop() {}
        
        // Deallocate memory
        unsafe {
            let layout = Layout::array::<T>(self.capacity).unwrap();
            dealloc(self.data as *mut u8, layout);
        }
    }
}

// Safe iterator for ring buffer
struct RingBufferIter<'a, T> {
    buffer: &'a RingBuffer<T>,
    index: usize,
}

impl<'a, T> Iterator for RingBufferIter<'a, T> {
    type Item = &'a T;
    
    fn next(&mut self) -> Option<Self::Item> {
        if self.index >= self.buffer.len {
            None
        } else {
            let actual_index = (self.buffer.head + self.index) % self.buffer.capacity;
            let item = unsafe {
                &*self.buffer.data.add(actual_index)
            };
            self.index += 1;
            Some(item)
        }
    }
}

// Memory pool for efficient allocation
struct MemoryPool<T> {
    blocks: Vec<*mut T>,
    block_size: usize,
    free_list: Vec<*mut T>,
}

impl<T> MemoryPool<T> {
    fn new(block_size: usize, initial_blocks: usize) -> Self {
        let mut pool = MemoryPool {
            blocks: Vec::new(),
            block_size,
            free_list: Vec::new(),
        };
        
        pool.allocate_block();
        
        for _ in 0..initial_blocks {
            pool.allocate_block();
        }
        
        pool
    }
    
    fn allocate_block(&mut self) {
        let layout = Layout::array::<T>(self.block_size).unwrap();
        
        unsafe {
            let block = alloc(layout) as *mut T;
            if block.is_null() {
                panic!("Failed to allocate memory block");
            }
            
            self.blocks.push(block);
            
            // Add all slots in this block to free list
            for i in 0..self.block_size {
                self.free_list.push(block.add(i));
            }
        }
    }
    
    fn allocate(&mut self) -> Option<*mut T> {
        if self.free_list.is_empty() {
            self.allocate_block();
        }
        
        self.free_list.pop()
    }
    
    fn deallocate(&mut self, ptr: *mut T) {
        // In a real implementation, we'd verify the pointer is valid
        self.free_list.push(ptr);
    }
    
    fn available_slots(&self) -> usize {
        self.free_list.len()
    }
    
    fn total_capacity(&self) -> usize {
        self.blocks.len() * self.block_size
    }
}

impl<T> Drop for MemoryPool<T> {
    fn drop(&mut self) {
        unsafe {
            let layout = Layout::array::<T>(self.block_size).unwrap();
            for block in &self.blocks {
                dealloc(*block as *mut u8, layout);
            }
        }
    }
}

// Safe wrapper for raw pointer operations
struct SafePtr<T> {
    ptr: *mut T,
    len: usize,
}

impl<T> SafePtr<T> {
    fn new(slice: &mut [T]) -> Self {
        SafePtr {
            ptr: slice.as_mut_ptr(),
            len: slice.len(),
        }
    }
    
    fn get(&self, index: usize) -> Option<&T> {
        if index < self.len {
            unsafe {
                Some(&*self.ptr.add(index))
            }
        } else {
            None
        }
    }
    
    fn get_mut(&mut self, index: usize) -> Option<&mut T> {
        if index < self.len {
            unsafe {
                Some(&mut *self.ptr.add(index))
            }
        } else {
            None
        }
    }
    
    fn len(&self) -> usize {
        self.len
    }
    
    // Unsafe but documented operation
    /// # Safety
    /// 
    /// The caller must ensure that:
    /// - `index` is less than `self.len()`
    /// - The pointer is still valid
    /// - No other mutable references exist to this element
    unsafe fn get_unchecked(&self, index: usize) -> &T {
        &*self.ptr.add(index)
    }
    
    /// # Safety
    /// 
    /// The caller must ensure that:
    /// - `index` is less than `self.len()`
    /// - The pointer is still valid
    /// - No other references exist to this element
    unsafe fn get_unchecked_mut(&mut self, index: usize) -> &mut T {
        &mut *self.ptr.add(index)
    }
}

fn safe_unsafe_patterns_demo() {
    println!("\n=== Safe Unsafe Code Patterns ===");
    
    // Ring buffer demonstration
    let mut ring = RingBuffer::new(3);
    
    println!("Ring buffer operations:");
    println!("  Initial: len={}, capacity={}", ring.len(), ring.capacity());
    
    // Fill the buffer
    ring.push("First").unwrap();
    ring.push("Second").unwrap();
    ring.push("Third").unwrap();
    
    println!("  After filling: len={}, is_full={}", ring.len(), ring.is_full());
    
    // Try to overfill
    if let Err(rejected) = ring.push("Fourth") {
        println!("  Rejected '{}' - buffer full", rejected);
    }
    
    // Iterate over contents
    print!("  Contents: ");
    for item in ring.iter() {
        print!("{} ", item);
    }
    println!();
    
    // Pop and push to test wraparound
    let first = ring.pop().unwrap();
    println!("  Popped: {}", first);
    
    ring.push("Fourth").unwrap();
    println!("  Added 'Fourth' after pop");
    
    print!("  New contents: ");
    for item in ring.iter() {
        print!("{} ", item);
    }
    println!();
    
    // Memory pool demonstration
    let mut pool: MemoryPool<i32> = MemoryPool::new(4, 2);
    
    println!("\nMemory pool operations:");
    println!("  Total capacity: {}", pool.total_capacity());
    println!("  Available slots: {}", pool.available_slots());
    
    // Allocate some memory
    let ptr1 = pool.allocate().unwrap();
    let ptr2 = pool.allocate().unwrap();
    let ptr3 = pool.allocate().unwrap();
    
    unsafe {
        *ptr1 = 42;
        *ptr2 = 84;
        *ptr3 = 126;
    }
    
    println!("  After allocating 3 slots: {} available", pool.available_slots());
    
    unsafe {
        println!("  Values: {} {} {}", *ptr1, *ptr2, *ptr3);
    }
    
    // Deallocate
    pool.deallocate(ptr2);
    println!("  After deallocating one: {} available", pool.available_slots());
    
    // Safe pointer wrapper
    let mut data = [1, 2, 3, 4, 5];
    let mut safe_ptr = SafePtr::new(&mut data);
    
    println!("\nSafe pointer operations:");
    println!("  Length: {}", safe_ptr.len());
    
    // Safe access
    if let Some(value) = safe_ptr.get(2) {
        println!("  safe_ptr[2] = {}", value);
    }
    
    // Safe mutation
    if let Some(value) = safe_ptr.get_mut(2) {
        *value = 99;
        println!("  Modified safe_ptr[2] to 99");
    }
    
    // Bounds checking
    if safe_ptr.get(10).is_none() {
        println!("  safe_ptr[10] is out of bounds (correctly rejected)");
    }
    
    // Unsafe but documented access
    unsafe {
        let value = safe_ptr.get_unchecked(1);
        println!("  Unsafe access safe_ptr[1] = {}", value);
    }
    
    println!("\n🛡️  Safety patterns demonstrated:");
    println!("  ✅ Bounds checking in safe wrappers");
    println!("  ✅ Proper memory management with Drop");
    println!("  ✅ Clear unsafe function documentation");
    println!("  ✅ Invariant maintenance across operations");
    println!("  ✅ Safe abstractions over unsafe primitives");
}

safe_unsafe_patterns_demo();

---

## 🧪 Active Recall Checkpoint

**Test your understanding without looking back:**

1. What are the five unsafe capabilities in Rust?
2. How do raw pointers differ from references?
3. What safety invariants must you maintain in unsafe code?
4. How does FFI work between Rust and C?
5. What is the purpose of #[no_mangle] and extern "C"?
6. How do you create safe abstractions around unsafe code?
7. What are unions and when would you use them?
8. How do you handle memory allocation in unsafe Rust?

**Write your answers below:**

**Your Answers:**
1. 
2. 
3. 
4. 
5. 
6. 
7. 
8. 

---

## 🤔 Reflection Prompt

Consider these questions:

1. **When is it appropriate to use unsafe Rust?**
2. **How do you balance performance and safety in systems programming?**
3. **What strategies help maintain safety invariants in unsafe code?**
4. **How does Rust's approach to unsafe code compare to other languages?**

Write your thoughts below:

**Your Reflections:**

1. 

2. 

3. 

4. 

---

## 🔮 Preview & Connections

### Coming Up Next: Performance & Optimization

In our next lesson, you'll learn about:
- Profiling and benchmarking Rust code
- Zero-cost abstractions and optimization
- Memory layout and cache optimization
- Performance best practices

### How This Connects
Unsafe Rust is essential for:
- Low-level systems programming
- Interfacing with C libraries
- Building safe abstractions
- Performance-critical code paths

---

## ✅ Expected Outcomes

**Self-Assessment Checklist** - Can you:

- [ ] Understand when unsafe Rust is necessary?
- [ ] Work with raw pointers safely?
- [ ] Implement FFI between Rust and C?
- [ ] Maintain safety invariants in unsafe code?
- [ ] Create safe abstractions around unsafe operations?
- [ ] Debug and test unsafe code effectively?
- [ ] Document unsafe code with proper safety comments?

If you checked all boxes, excellent! You've mastered unsafe Rust and FFI.

---

**🎉 Masterful Achievement!** You now understand how to safely work with unsafe Rust and foreign code!